This lab on Principal Components Analysis in R is an adaptation of p. 401-404,
408-410 of "Introduction to Statistical Learning with Applications in R" by Gareth James,
Daniela Witten, Trevor Hastie and Robert Tibshirani. Adapted by R. Jordan Crouser at
Smith College for SDS293: Machine Learning (Spring 2016).

# 10.4 Lab 1: Principal Components Analysis

In this lab, we perform PCA on the ${\tt USArrests}$ data set, which is part of
the base ${\tt R}$ package. The rows of the data set contain the 50 states, in
alphabetical order:

In [ ]:
states=row.names(USArrests)
states

The columns of the data set contain four variables relating to various crimes:

In [ ]:
names(USArrests)

Let's start by taking a quick look at the column means of the data. We can use the ${\tt apply()}$ function to apply a function - in this case, the ${\tt mean()}$ function - to each row or column of the data set. The second
input here denotes whether we wish to compute the mean of the rows, 1,
or the columns, 2:

In [ ]:
apply(USArrests, 2, mean)

We see right away the the data have **vastly** different means. We can also examine the variances of the four variables using the ${\tt apply()}$ function:

In [ ]:
apply(USArrests, 2, var)

Not surprisingly, the variables also have vastly different variances: the
${\tt UrbanPop}$ variable measures the percentage of the population in each state
living in an urban area, which is not a comparable number to the number
of crimes committeed in each state per 100,000 individuals. If we failed to scale the
variables before performing PCA, then most of the principal components
that we observed would be driven by the ${\tt Assault}$ variable, since it has by
far the largest mean and variance. 

Thus, it is important to standardize the
variables to have mean zero and standard deviation 1 before performing
PCA. In this lab, we'll perform principal components analysis using the ${\tt prcomp()}$ function, which is one of several functions in ${\tt R}$ that perform PCA. By default, the ${\tt prcomp()}$ function centers the variables to have mean zero. By using the option ${\tt scale=TRUE}$, we scale the variables to have standard
deviation 1:

In [ ]:
pr.out=prcomp(USArrests, scale=TRUE)

The output from ${\tt prcomp()}$ contains a number of useful quantities:

In [ ]:
names(pr.out)

The ${\tt center}$ and ${\tt scale}$ components correspond to the means and standard
deviations of the variables that were used for scaling prior to implementing
PCA:

In [ ]:
pr.out$center
pr.out$scale

The rotation matrix provides the principal component loadings; each column
of ${\tt pr.out\$rotation}$ contains the corresponding principal component
loading vector:

In [ ]:
pr.out$rotation

We see that there are four distinct principal components. This is to be
expected because there are in general ${\tt min(n − 1, p)}$ informative principal
components in a data set with $n$ observations and $p$ variables.

Using the ${\tt prcomp()}$ function, we do not need to explicitly multiply the
data by the principal component loading vectors in order to obtain the
principal component score vectors. Rather the 50 × 4 matrix $x$ has as its
columns the principal component score vectors. That is, the $k^{th}$ column is
the $k^{th}$ principal component score vector. We'll take a look at the first few states:

In [ ]:
head(pr.out$x)

We can plot the first two principal components using the ${\tt biplot()}$ function:

In [ ]:
biplot(pr.out, scale=0)

The ${\tt scale=0}$ argument to ${\tt biplot()}$ ensures that the arrows are scaled to
represent the loadings; other values for ${\tt scale}$ give slightly different biplots
with different interpretations.

The ${\tt prcomp()}$ function also outputs the standard deviation of each principal
component. We can access these standard deviations as follows:

In [ ]:
pr.out$sdev

The variance explained by each principal component is obtained by squaring
these:

In [ ]:
pr.var=pr.out$sdev^2
pr.var

To compute the proportion of variance explained by each principal component,
we simply divide the variance explained by each principal component
by the total variance explained by all four principal components:

In [ ]:
pve=pr.var/sum(pr.var)
pve

We see that the first principal component explains 62.0% of the variance
in the data, the next principal component explains 24.7% of the variance,
and so forth. We can plot the PVE explained by each component as follows:

In [ ]:
plot(pve, xlab="Principal Component", ylab="Proportion of Variance Explained", ylim=c(0,1),type='b')

We can also use the function ${\tt cumsum()}$, which computes the cumulative sum of the elements of a numeric vector, to plot the cumulative PVE:

In [ ]:
plot(cumsum(pve), xlab="Principal Component", ylab="Cumulative Proportion of Variance Explained", ylim=c(0,1),type='b')
a=c(1,2,8,-3)
cumsum(a)

# 10.6: NCI60 Data Example

Let's return to the ${\tt NCI60}$ cancer cell line microarray data, which
consists of 6,830 gene expression measurements on 64 cancer cell lines:

In [ ]:
library(ISLR)
nci.labs=NCI60$labs
nci.data=NCI60$data

# 10.6.1 PCA on the NCI60 Data

We first perform PCA on the data after scaling the variables (genes) to
have standard deviation one, although one could reasonably argue that it
is better not to scale the genes:

In [ ]:
pr.out=prcomp(nci.data, scale=TRUE)

We now plot the first few principal component score vectors, in order to
visualize the data. The observations (cell lines) corresponding to a given
cancer type will be plotted in the same color, so that we can see to what
extent the observations within a cancer type are similar to each other. We
first create a simple function that assigns a distinct color to each element
of a numeric vector. The function will be used to assign a color to each of
the 64 cell lines, based on the cancer type to which it corresponds.
We'll make use of the ${\tt rainbow()}$ function, which takes as its argument a positive integer,
and returns a vector containing that number of distinct colors.

In [ ]:
Cols=function(vec){
    cols=rainbow(length(unique(vec)))
    return(cols[as.numeric(as.factor(vec))])
  }

We now can plot the principal component score vectors:

In [ ]:
par(mfrow=c(1,2))
plot(pr.out$x[,1:2], col=Cols(nci.labs), pch=19,xlab="Z1",ylab="Z2")
plot(pr.out$x[,c(1,3)], col=Cols(nci.labs), pch=19,xlab="Z1",ylab="Z3")

On the whole, cell lines corresponding to a single cancer type do tend to have similar values on the
first few principal component score vectors. This indicates that cell lines
from the same cancer type tend to have pretty similar gene expression
levels.

We can obtain a summary of the proportion of variance explained (PVE)
of the first few principal components using the ${\tt summary()}$ method for a
${\tt prcomp}$ object:

In [ ]:
summary(pr.out)

Using the ${\tt plot()}$ function, we can also plot the variance explained by the
first few principal components:

In [ ]:
plot(pr.out)

Note that the height of each bar in the bar plot is given by squaring the
corresponding element of ${\tt pr.out\$sdev}$. However, it is generally more informative to
plot the PVE of each principal component (i.e. a **scree plot**) and the cumulative
PVE of each principal component. This can be done with just a
little tweaking:

In [ ]:
pve=100*pr.out$sdev^2/sum(pr.out$sdev^2)
par(mfrow=c(1,2))
plot(pve,  type="o", ylab="PVE", xlab="Principal Component", col="blue")
plot(cumsum(pve), type="o", ylab="Cumulative PVE", xlab="Principal Component", col="brown3")

We see that together, the first seven principal components
explain around 40% of the variance in the data. This is not a huge amount
of the variance. However, looking at the scree plot, we see that while each
of the first seven principal components explain a substantial amount of
variance, there is a marked decrease in the variance explained by further
principal components. That is, there is an **elbow** in the plot after approximately
the seventh principal component. This suggests that there may
be little benefit to examining more than seven or so principal components
(phew! even examining seven principal components may be difficult).